In [0]:
# Airflow Integration Parameters

try:
    # Create widgets for Airflow parameters
    dbutils.widgets.text("batch_id", "manual_run", "Batch ID from Airflow")
    dbutils.widgets.text("execution_date", "", "Execution Date from Airflow") 
    dbutils.widgets.text("force_refresh", "false", "Force data refresh")
    dbutils.widgets.text("quality_threshold", "0.8", "Data quality threshold")
    dbutils.widgets.text("dag_run_id", "", "DAG Run ID")
    
    # Get parameter values
    batch_id = dbutils.widgets.get("batch_id")
    execution_date = dbutils.widgets.get("execution_date")
    force_refresh = dbutils.widgets.get("force_refresh").lower() == "true"
    quality_threshold = float(dbutils.widgets.get("quality_threshold"))
    dag_run_id = dbutils.widgets.get("dag_run_id")
    
    print(f"🎯 Airflow Parameters:")
    print(f"   Batch ID: {batch_id}")
    print(f"   Execution Date: {execution_date}")
    print(f"   Force Refresh: {force_refresh}")
    print(f"   Quality Threshold: {quality_threshold}")
    print(f"   DAG Run ID: {dag_run_id}")
    
except Exception as e:
    print(f"⚠️ Widget creation failed (normal in some contexts): {e}")
    # Fallback values for manual runs
    batch_id = "manual_run"
    execution_date = ""
    force_refresh = False
    quality_threshold = 0.8
    dag_run_id = ""

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC **Enhanced Analytics with Explicit Schema and FinBERT Sentiment Analysis**
# MAGIC 
# MAGIC This notebook combines the best features from both approaches:
# MAGIC - Explicit schema definitions for type safety and DataFrame creation stability
# MAGIC - Complete pipeline with comprehensive technical indicators
# MAGIC - Production-grade FinBERT sentiment analysis with fallback mechanisms
# MAGIC - Advanced quality scoring and correlation readiness
# MAGIC - Full verification and optimization suite

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Setup and Configuration with Enhanced Error Handling

# COMMAND ----------

# Store Python's built-in functions before PySpark imports override them
import builtins
python_round = builtins.round
python_min = builtins.min
python_max = builtins.max
python_abs = builtins.abs

# Set environment variables BEFORE any torch imports
import os
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '12355' 
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import time
import re
from datetime import datetime, timedelta, timezone
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import traceback

# Import torch with distributed training disabled
import torch
torch.distributed.is_available = lambda: False
torch.distributed.is_initialized = lambda: False

print("🥈 Silver Layer Processor - Complete Combined Version")
print("=" * 60)
print(f"⏰ Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Focus: Bronze → Silver with explicit schemas and comprehensive analytics")

# COMMAND ----------

# Enhanced Configuration with catalog awareness
try:
    # Get current catalog for table references
    current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
    print(f"📋 Current catalog: {current_catalog}")
    
    # Table references with catalog context
    bronze_stock_table = f"{current_catalog}.bronze.historical_stock_data"
    bronze_news_table = f"{current_catalog}.bronze.historical_news_data"
    silver_stock_table = f"{current_catalog}.silver.enhanced_stock_data"
    silver_news_table = f"{current_catalog}.silver.enhanced_news_data"
    silver_metrics_table = f"{current_catalog}.silver.processing_metrics"
    
    print("✅ Configuration loaded successfully")
    print(f"📊 Bronze Stock Table: {bronze_stock_table}")
    print(f"📰 Bronze News Table: {bronze_news_table}")
    
except Exception as e:
    print(f"❌ Configuration error: {e}")
    raise

# Enhanced processing configuration
SILVER_CONFIG = {
    "batch_size": 1000,
    "finbert_confidence_threshold": 0.6,
    "data_quality_threshold": 0.7,
    "technical_indicators_window": 20,
    "sentiment_aggregation_window": 7,
    "correlation_analysis_enabled": True,
    "feature_engineering_enabled": True,
    "max_news_records_per_batch": 50,  # Limit for stable processing
    "enable_explicit_schema": True
}

print(f"📊 Enhanced processing configuration:")
for key, value in SILVER_CONFIG.items():
    print(f"   {key}: {value}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Explicit Schema Definitions for Type Safety

# COMMAND ----------

def get_enhanced_stock_schema():
    """Define comprehensive schema for enhanced stock data"""
    return StructType([
        # Basic stock data
        StructField("symbol", StringType(), True),
        StructField("date", DateType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("open_price", DoubleType(), True),
        StructField("high_price", DoubleType(), True),
        StructField("low_price", DoubleType(), True),
        StructField("close_price", DoubleType(), True),
        StructField("adjusted_close", DoubleType(), True),
        StructField("volume", LongType(), True),
        StructField("split_coefficient", DoubleType(), True),
        StructField("dividend_amount", DoubleType(), True),
        
        # Price movement indicators
        StructField("price_change", DoubleType(), True),
        StructField("price_change_pct", DoubleType(), True),
        StructField("daily_range", DoubleType(), True),
        StructField("daily_range_pct", DoubleType(), True),
        StructField("gap_pct", DoubleType(), True),
        
        # Volume indicators
        StructField("volume_change", LongType(), True),
        StructField("volume_change_pct", DoubleType(), True),
        StructField("volume_ma_5d", DoubleType(), True),
        StructField("volume_ma_20d", DoubleType(), True),
        StructField("volume_ratio", DoubleType(), True),
        
        # Moving averages
        StructField("sma_5d", DoubleType(), True),
        StructField("sma_10d", DoubleType(), True),
        StructField("sma_20d", DoubleType(), True),
        StructField("sma_50d", DoubleType(), True),
        StructField("ema_12d", DoubleType(), True),
        StructField("ema_26d", DoubleType(), True),
        
        # Technical indicators
        StructField("rsi_14d", DoubleType(), True),
        StructField("macd", DoubleType(), True),
        StructField("macd_signal", DoubleType(), True),
        StructField("macd_histogram", DoubleType(), True),
        StructField("bollinger_upper", DoubleType(), True),
        StructField("bollinger_lower", DoubleType(), True),
        StructField("bollinger_position", DoubleType(), True),
        
        # Volatility measures
        StructField("volatility_5d", DoubleType(), True),
        StructField("volatility_20d", DoubleType(), True),
        StructField("atr_14d", DoubleType(), True),
        
        # Trend analysis
        StructField("trend_direction", StringType(), True),
        StructField("trend_strength", DoubleType(), True),
        StructField("support_level", DoubleType(), True),
        StructField("resistance_level", DoubleType(), True),
        
        # Signal generation
        StructField("technical_signal", StringType(), True),
        StructField("market_sentiment", StringType(), True),
        StructField("momentum_indicator", StringType(), True),
        
        # Quality metrics
        StructField("data_quality_score", DoubleType(), True),
        StructField("completeness_score", DoubleType(), True),
        StructField("reliability_score", DoubleType(), True),
        
        # Processing metadata
        StructField("source", StringType(), True),
        StructField("processing_timestamp", TimestampType(), True),
        StructField("ingestion_batch", StringType(), True),
        StructField("processed_date", DateType(), True)
    ])

def get_enhanced_news_schema():
    """Define comprehensive schema for enhanced news data with FinBERT"""
    return StructType([
        # Basic news data
        StructField("article_id", StringType(), True),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("content", StringType(), True),
        StructField("url", StringType(), True),
        StructField("source", StringType(), True),
        StructField("author", StringType(), True),
        StructField("published_at", TimestampType(), True),
        StructField("published_date", DateType(), True),
        
        # Content metrics
        StructField("content_length", IntegerType(), True),
        StructField("title_length", IntegerType(), True),
        StructField("readability_score", DoubleType(), True),
        StructField("financial_relevance_score", DoubleType(), True),
        
        # Entity extraction
        StructField("mentioned_symbols", ArrayType(StringType()), True),
        StructField("financial_entities", ArrayType(StringType()), True),
        StructField("key_financial_terms", ArrayType(StringType()), True),
        StructField("named_entities", ArrayType(StringType()), True),
        
        # Original sentiment indicators
        StructField("original_sentiment_indicators", MapType(StringType(), DoubleType()), True),
        
        # FinBERT sentiment analysis
        StructField("finbert_label", StringType(), True),
        StructField("finbert_score", DoubleType(), True),
        StructField("finbert_confidence", DoubleType(), True),
        StructField("finbert_negative", DoubleType(), True),
        StructField("finbert_neutral", DoubleType(), True),
        StructField("finbert_positive", DoubleType(), True),
        StructField("finbert_processing_method", StringType(), True),
        StructField("finbert_processing_time_ms", DoubleType(), True),
        
        # Enhanced sentiment features
        StructField("sentiment_intensity", StringType(), True),
        StructField("sentiment_subjectivity", DoubleType(), True),
        StructField("emotional_tone", StringType(), True),
        StructField("urgency_score", DoubleType(), True),
        
        # Categorization
        StructField("news_category", StringType(), True),
        StructField("market_impact_category", StringType(), True),
        StructField("time_sensitivity", StringType(), True),
        
        # Quality scoring
        StructField("content_quality_score", DoubleType(), True),
        StructField("sentiment_quality_score", DoubleType(), True),
        StructField("overall_reliability_score", DoubleType(), True),
        
        # Temporal features
        StructField("market_hours_flag", BooleanType(), True),
        StructField("weekday_flag", BooleanType(), True),
        StructField("earnings_season_flag", BooleanType(), True),
        
        # Processing metadata
        StructField("processing_timestamp", TimestampType(), True),
        StructField("data_source", StringType(), True),
        StructField("ingestion_batch", StringType(), True),
        StructField("processed_date", DateType(), True)
    ])

print("✅ Explicit schemas defined for type-safe DataFrame creation")
print(f"📊 Stock schema: {len(get_enhanced_stock_schema())} fields")
print(f"📰 News schema: {len(get_enhanced_news_schema())} fields")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Enhanced FinBERT

# COMMAND ----------

# Load FinBERT with comprehensive error handling
finbert_ready = False
processing_stats = {"finbert_calls": 0, "total_time": 0.0, "avg_time_per_call": 0.0}

try:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import torch
    import numpy as np
    
    print("🤖 Loading enhanced FinBERT model...")
    
    # Force CPU-only mode
    device = torch.device('cpu')
    
    # Load tokenizer with explicit settings
    tokenizer = AutoTokenizer.from_pretrained(
        "ProsusAI/finbert",
        use_fast=True,
        local_files_only=False,
        trust_remote_code=False
    )
    
    # Load model with CPU-only configuration
    model = AutoModelForSequenceClassification.from_pretrained(
        "ProsusAI/finbert",
        torch_dtype=torch.float32,
        device_map=None,
        low_cpu_mem_usage=True,
        trust_remote_code=False
    )
    
    # Move to CPU and disable gradients
    model.to(device)
    model.eval()
    torch.set_grad_enabled(False)
    
    finbert_ready = True
    print("✅ FinBERT model ready for production")
    
except Exception as e:
    print(f"⚠️ FinBERT loading failed, using enhanced fallback: {e}")
    finbert_ready = False

def enhanced_financial_preprocessing(text):
    """Enhanced preprocessing for financial text"""
    if not text or not isinstance(text, str):
        return "No content available"
    
    # Handle complex financial sentences
    text = re.sub(r',(\\s+(?:but|although|while|however))', r' \\1', text)
    
    # Focus on sentiment-bearing parts after conjunctions
    sentiment_markers = ['but', 'however', 'although', 'despite', 'yet', 'while', 'nevertheless']
    for marker in sentiment_markers:
        if marker in text.lower():
            parts = text.lower().split(marker)
            if len(parts) > 1:
                text = parts[-1].strip()
                break
    
    # Clean financial patterns
    text = re.sub(r'\\$\\s*\\d+(?:\\.\\d+)?[BMK]?', '[AMOUNT]', text)
    text = re.sub(r'\\d+\\.\\d+%', '[PERCENT]', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    
    return text[:512]

def apply_production_finbert_sentiment(text):
    """Production-grade FinBERT sentiment analysis with comprehensive fallback"""
    
    global processing_stats
    start_time = time.time()
    
    if not finbert_ready:
        # Enhanced fallback sentiment analysis
        text_lower = text.lower()
        
        sentiment_weights = {
            'strong_positive': {
                'words': ['surge', 'soars', 'beats', 'exceeds', 'outperforms', 'bullish', 'rally', 'breakthrough', 'skyrockets'],
                'weight': 0.4
            },
            'positive': {
                'words': ['gains', 'rises', 'up', 'growth', 'strong', 'positive', 'improved', 'boost', 'optimistic', 'upbeat'],
                'weight': 0.2
            },
            'strong_negative': {
                'words': ['plummet', 'crashes', 'tanks', 'collapses', 'bearish', 'panic', 'crisis', 'plunges', 'devastated'],
                'weight': -0.4
            },
            'negative': {
                'words': ['falls', 'drops', 'declines', 'weak', 'negative', 'pressure', 'concerns', 'pessimistic', 'worried'],
                'weight': -0.2
            }
        }
        
        total_score = 0.0
        word_count = 0
        
        for category, data in sentiment_weights.items():
            matches = sum(1 for word in data['words'] if word in text_lower)
            total_score += matches * data['weight']
            word_count += matches
        
        sentiment_score = python_max(-1.0, python_min(1.0, total_score)) if word_count > 0 else 0.0
        
        if sentiment_score > 0.2:
            label = 'positive'
            confidence = python_min(0.9, 0.6 + python_abs(sentiment_score))
        elif sentiment_score < -0.2:
            label = 'negative' 
            confidence = python_min(0.9, 0.6 + python_abs(sentiment_score))
        else:
            label = 'neutral'
            confidence = 0.7
        
        processing_time = time.time() - start_time
        processing_stats["finbert_calls"] += 1
        processing_stats["total_time"] += processing_time
        
        return {
            'finbert_label': label,
            'finbert_score': sentiment_score,
            'finbert_confidence': confidence,
            'finbert_negative': 0.5 - sentiment_score/2 if sentiment_score <= 0 else 0.1,
            'finbert_neutral': 0.5 - python_abs(sentiment_score)/2,
            'finbert_positive': 0.5 + sentiment_score/2 if sentiment_score >= 0 else 0.1,
            'processing_method': 'enhanced_fallback',
            'processing_time_ms': processing_time * 1000
        }
    
    try:
        clean_text = enhanced_financial_preprocessing(text)
        
        if not clean_text or clean_text == "No content available":
            processing_time = time.time() - start_time
            processing_stats["finbert_calls"] += 1
            processing_stats["total_time"] += processing_time
            
            return {
                'finbert_label': 'neutral',
                'finbert_score': 0.0,
                'finbert_confidence': 0.5,
                'finbert_negative': 0.33,
                'finbert_neutral': 0.34,
                'finbert_positive': 0.33,
                'processing_method': 'empty_text',
                'processing_time_ms': processing_time * 1000
            }
        
        # CPU-only tokenization
        inputs = tokenizer(
            clean_text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )
        
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        predictions = predictions.cpu().numpy()[0]
        labels = ['negative', 'neutral', 'positive']
        
        max_idx = np.argmax(predictions)
        predicted_label = labels[max_idx]
        confidence = float(predictions[max_idx])
        
        if predicted_label == 'positive':
            score = confidence
        elif predicted_label == 'negative':
            score = -confidence
        else:
            score = float(predictions[2]) - float(predictions[0])
        
        processing_time = time.time() - start_time
        processing_stats["finbert_calls"] += 1
        processing_stats["total_time"] += processing_time
        processing_stats["avg_time_per_call"] = processing_stats["total_time"] / processing_stats["finbert_calls"]
        
        return {
            'finbert_label': predicted_label,
            'finbert_score': python_round(score, 4),
            'finbert_confidence': python_round(confidence, 4),
            'finbert_negative': python_round(float(predictions[0]), 4),
            'finbert_neutral': python_round(float(predictions[1]), 4),
            'finbert_positive': python_round(float(predictions[2]), 4),
            'processing_method': 'finbert_production',
            'processing_time_ms': python_round(processing_time * 1000, 2)
        }
        
    except Exception as e:
        processing_time = time.time() - start_time
        processing_stats["finbert_calls"] += 1
        processing_stats["total_time"] += processing_time
        
        return {
            'finbert_label': 'neutral',
            'finbert_score': 0.0,
            'finbert_confidence': 0.5,
            'finbert_negative': 0.33,
            'finbert_neutral': 0.34,
            'finbert_positive': 0.33,
            'processing_method': 'error_fallback',
            'processing_time_ms': processing_time * 1000
        }

print(f"🤖 FinBERT Status: {'Production Ready' if finbert_ready else 'Enhanced Fallback'}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Create Enhanced Silver Layer Tables

# COMMAND ----------

def create_enhanced_silver_tables():
    """Create Silver layer tables with explicit schemas"""
    
    print("🏗️ Creating enhanced Silver layer tables...")
    
    # Create silver schema
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {current_catalog}.silver")
    print(f"✅ Silver schema created in catalog: {current_catalog}")
    
    # Enhanced Stock Data Table
    stock_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {silver_stock_table} (
            symbol STRING,
            date DATE,
            timestamp TIMESTAMP,
            open_price DOUBLE,
            high_price DOUBLE,
            low_price DOUBLE,
            close_price DOUBLE,
            adjusted_close DOUBLE,
            volume BIGINT,
            split_coefficient DOUBLE,
            dividend_amount DOUBLE,
            
            price_change DOUBLE,
            price_change_pct DOUBLE,
            daily_range DOUBLE,
            daily_range_pct DOUBLE,
            gap_pct DOUBLE,
            
            volume_change BIGINT,
            volume_change_pct DOUBLE,
            volume_ma_5d DOUBLE,
            volume_ma_20d DOUBLE,
            volume_ratio DOUBLE,
            
            sma_5d DOUBLE,
            sma_10d DOUBLE,
            sma_20d DOUBLE,
            sma_50d DOUBLE,
            ema_12d DOUBLE,
            ema_26d DOUBLE,
            
            rsi_14d DOUBLE,
            macd DOUBLE,
            macd_signal DOUBLE,
            macd_histogram DOUBLE,
            bollinger_upper DOUBLE,
            bollinger_lower DOUBLE,
            bollinger_position DOUBLE,
            
            volatility_5d DOUBLE,
            volatility_20d DOUBLE,
            atr_14d DOUBLE,
            
            trend_direction STRING,
            trend_strength DOUBLE,
            support_level DOUBLE,
            resistance_level DOUBLE,
            
            technical_signal STRING,
            market_sentiment STRING,
            momentum_indicator STRING,
            
            data_quality_score DOUBLE,
            completeness_score DOUBLE,
            reliability_score DOUBLE,
            source STRING,
            processing_timestamp TIMESTAMP,
            ingestion_batch STRING,
            processed_date DATE
        ) USING DELTA
        PARTITIONED BY (processed_date, symbol)
        TBLPROPERTIES (
            'delta.autoOptimize.optimizeWrite' = 'true',
            'delta.autoOptimize.autoCompact' = 'true'
        )
    """
    
    spark.sql(stock_table_sql)
    print(f"✅ Created enhanced stock table: {silver_stock_table}")
    
    # Enhanced News Data Table
    news_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {silver_news_table} (
            article_id STRING,
            title STRING,
            description STRING,
            content STRING,
            url STRING,
            source STRING,
            author STRING,
            published_at TIMESTAMP,
            published_date DATE,
            
            content_length INT,
            title_length INT,
            readability_score DOUBLE,
            financial_relevance_score DOUBLE,
            
            mentioned_symbols ARRAY<STRING>,
            financial_entities ARRAY<STRING>,
            key_financial_terms ARRAY<STRING>,
            named_entities ARRAY<STRING>,
            
            original_sentiment_indicators MAP<STRING, DOUBLE>,
            
            finbert_label STRING,
            finbert_score DOUBLE,
            finbert_confidence DOUBLE,
            finbert_negative DOUBLE,
            finbert_neutral DOUBLE,
            finbert_positive DOUBLE,
            finbert_processing_method STRING,
            finbert_processing_time_ms DOUBLE,
            
            sentiment_intensity STRING,
            sentiment_subjectivity DOUBLE,
            emotional_tone STRING,
            urgency_score DOUBLE,
            
            news_category STRING,
            market_impact_category STRING,
            time_sensitivity STRING,
            
            content_quality_score DOUBLE,
            sentiment_quality_score DOUBLE,
            overall_reliability_score DOUBLE,
            
            market_hours_flag BOOLEAN,
            weekday_flag BOOLEAN,
            earnings_season_flag BOOLEAN,
            
            processing_timestamp TIMESTAMP,
            data_source STRING,
            ingestion_batch STRING,
            processed_date DATE
        ) USING DELTA
        PARTITIONED BY (processed_date, finbert_label)
        TBLPROPERTIES (
            'delta.autoOptimize.optimizeWrite' = 'true',
            'delta.autoOptimize.autoCompact' = 'true'
        )
    """
    
    spark.sql(news_table_sql)
    print(f"✅ Created enhanced news table: {silver_news_table}")
    
    print("✅ Enhanced Silver layer tables created successfully")

create_enhanced_silver_tables()

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Comprehensive Technical Indicators

# COMMAND ----------

def calculate_comprehensive_technical_indicators(stock_df):
    """Calculate comprehensive technical indicators with enhanced stability"""
    
    print("📊 Calculating comprehensive technical indicators...")
    
    # Define window specifications
    windows = {
        'w5': Window.partitionBy("symbol").orderBy("date").rowsBetween(-4, 0),
        'w10': Window.partitionBy("symbol").orderBy("date").rowsBetween(-9, 0),
        'w20': Window.partitionBy("symbol").orderBy("date").rowsBetween(-19, 0),
        'w50': Window.partitionBy("symbol").orderBy("date").rowsBetween(-49, 0),
        'w14': Window.partitionBy("symbol").orderBy("date").rowsBetween(-13, 0),
        'lag': Window.partitionBy("symbol").orderBy("date")
    }
    
    # Ensure proper data types
    stock_df = stock_df.withColumn("close_price", col("close_price").cast("double")) \
                     .withColumn("open_price", col("open_price").cast("double")) \
                     .withColumn("high_price", col("high_price").cast("double")) \
                     .withColumn("low_price", col("low_price").cast("double")) \
                     .withColumn("volume", col("volume").cast("bigint")) \
                     .withColumn("date", col("date").cast("date")) \
                     .withColumn("timestamp", col("timestamp").cast("timestamp"))
    
    # Basic price calculations with null handling
    enhanced_df = stock_df.withColumn(
        "price_change",
        coalesce(col("close_price") - lag("close_price").over(windows['lag']), lit(0.0))
    ).withColumn(
        "price_change_pct",
        when(coalesce(lag("close_price").over(windows['lag']), lit(0)) != 0,
             (col("close_price") - lag("close_price").over(windows['lag'])) / 
             lag("close_price").over(windows['lag']) * 100
        ).otherwise(0.0)
    ).withColumn(
        "daily_range",
        coalesce(col("high_price") - col("low_price"), lit(0.0))
    ).withColumn(
        "daily_range_pct",
        when(coalesce(col("close_price"), lit(0)) != 0,
             (col("high_price") - col("low_price")) / col("close_price") * 100
        ).otherwise(0.0)
    ).withColumn(
        "gap_pct",
        when(coalesce(lag("close_price").over(windows['lag']), lit(0)) != 0,
             (col("open_price") - lag("close_price").over(windows['lag'])) / 
             lag("close_price").over(windows['lag']) * 100
        ).otherwise(0.0)
    )
    
    # Volume calculations
    volume_df = enhanced_df.withColumn(
        "volume_change",
        coalesce(col("volume") - lag("volume").over(windows['lag']), lit(0))
    ).withColumn(
        "volume_change_pct",
        when(coalesce(lag("volume").over(windows['lag']), lit(0)) != 0,
             (col("volume") - lag("volume").over(windows['lag'])) / 
             lag("volume").over(windows['lag']).cast("double") * 100
        ).otherwise(0.0)
    ).withColumn(
        "volume_ma_5d",
        coalesce(avg("volume").over(windows['w5']), lit(0.0))
    ).withColumn(
        "volume_ma_20d",
        coalesce(avg("volume").over(windows['w20']), lit(0.0))
    ).withColumn(
        "volume_ratio",
        when(coalesce(avg("volume").over(windows['w20']), lit(0)) != 0,
             col("volume").cast("double") / avg("volume").over(windows['w20'])
        ).otherwise(1.0)
    )
    
    # Moving averages with null handling
    ma_df = volume_df.withColumn(
        "sma_5d",
        coalesce(avg("close_price").over(windows['w5']), lit(0.0))
    ).withColumn(
        "sma_10d",
        coalesce(avg("close_price").over(windows['w10']), lit(0.0))
    ).withColumn(
        "sma_20d",
        coalesce(avg("close_price").over(windows['w20']), lit(0.0))
    ).withColumn(
        "sma_50d",
        coalesce(avg("close_price").over(windows['w50']), lit(0.0))
    )
    
    # RSI calculation with stability
    rsi_df = ma_df.withColumn(
        "gain",
        when(coalesce(col("price_change"), lit(0)) > 0, col("price_change")).otherwise(0)
    ).withColumn(
        "loss",
        when(coalesce(col("price_change"), lit(0)) < 0, -col("price_change")).otherwise(0)
    ).withColumn(
        "avg_gain",
        coalesce(avg("gain").over(windows['w14']), lit(0.0))
    ).withColumn(
        "avg_loss",
        coalesce(avg("loss").over(windows['w14']), lit(0.0))
    ).withColumn(
        "rsi_14d",
        when(coalesce(col("avg_loss"), lit(0)) > 0, 
             100 - (100 / (1 + col("avg_gain") / col("avg_loss"))))
        .otherwise(50.0)
    )
    
    # Enhanced MACD and Bollinger Bands
    enhanced_indicators_df = rsi_df.withColumn(
        "ema_12d",
        coalesce(avg("close_price").over(Window.partitionBy("symbol").orderBy("date").rowsBetween(-11, 0)), lit(0.0))
    ).withColumn(
        "ema_26d",
        coalesce(avg("close_price").over(Window.partitionBy("symbol").orderBy("date").rowsBetween(-25, 0)), lit(0.0))
    ).withColumn(
        "macd",
        coalesce(col("ema_12d") - col("ema_26d"), lit(0.0))
    ).withColumn(
        "macd_signal",
        coalesce(avg("macd").over(Window.partitionBy("symbol").orderBy("date").rowsBetween(-8, 0)), lit(0.0))
    ).withColumn(
        "macd_histogram",
        coalesce(col("macd") - col("macd_signal"), lit(0.0))
    ).withColumn(
        "bollinger_upper",
        coalesce(col("sma_20d") + (2 * coalesce(stddev("close_price").over(windows['w20']), lit(0))), col("sma_20d"))
    ).withColumn(
        "bollinger_lower",
        coalesce(col("sma_20d") - (2 * coalesce(stddev("close_price").over(windows['w20']), lit(0))), col("sma_20d"))
    ).withColumn(
        "bollinger_position",
        when((col("bollinger_upper") - col("bollinger_lower")) != 0,
             (col("close_price") - col("bollinger_lower")) / 
             (col("bollinger_upper") - col("bollinger_lower"))
        ).otherwise(0.5)
    )
    
    # Additional volatility and support/resistance
    volatility_df = enhanced_indicators_df.withColumn(
        "volatility_5d",
        coalesce(stddev("price_change_pct").over(windows['w5']), lit(0.0))
    ).withColumn(
        "volatility_20d",
        coalesce(stddev("price_change_pct").over(windows['w20']), lit(0.0))
    ).withColumn(
        "atr_14d",
        coalesce(avg(greatest(
            col("high_price") - col("low_price"),
            abs(col("high_price") - coalesce(lag("close_price").over(windows['lag']), col("close_price"))),
            abs(col("low_price") - coalesce(lag("close_price").over(windows['lag']), col("close_price")))
        )).over(windows['w14']), lit(0.0))
    ).withColumn(
        "support_level",
        coalesce(min("low_price").over(windows['w20']), lit(0.0))
    ).withColumn(
        "resistance_level",
        coalesce(max("high_price").over(windows['w20']), lit(0.0))
    )
    
    # Technical signals and quality scoring
    final_df = volatility_df.withColumn(
        "trend_direction",
        when(coalesce(col("sma_5d"), lit(0)) > coalesce(col("sma_20d"), lit(0)), "uptrend")
        .when(coalesce(col("sma_5d"), lit(0)) < coalesce(col("sma_20d"), lit(0)), "downtrend")
        .otherwise("sideways")
    ).withColumn(
        "trend_strength",
        when(coalesce(col("sma_20d"), lit(0)) != 0,
             abs(col("sma_5d") - col("sma_20d")) / col("sma_20d")
        ).otherwise(0.0)
    ).withColumn(
        "technical_signal",
        when((coalesce(col("rsi_14d"), lit(50)) < 30) & (col("close_price") < col("bollinger_lower")), "strong_buy")
        .when((coalesce(col("rsi_14d"), lit(50)) < 40) & (col("trend_direction") == "uptrend"), "buy")
        .when((coalesce(col("rsi_14d"), lit(50)) > 70) & (col("close_price") > col("bollinger_upper")), "strong_sell")
        .when((coalesce(col("rsi_14d"), lit(50)) > 60) & (col("trend_direction") == "downtrend"), "sell")
        .otherwise("hold")
    ).withColumn(
        "market_sentiment",
        when((coalesce(col("volume_ratio"), lit(1)) > 2) & (coalesce(col("price_change_pct"), lit(0)) > 3), "strong_bullish")
        .when((coalesce(col("volume_ratio"), lit(1)) > 1.5) & (coalesce(col("price_change_pct"), lit(0)) > 1), "bullish")
        .when((coalesce(col("volume_ratio"), lit(1)) > 2) & (coalesce(col("price_change_pct"), lit(0)) < -3), "strong_bearish")
        .when((coalesce(col("volume_ratio"), lit(1)) > 1.5) & (coalesce(col("price_change_pct"), lit(0)) < -1), "bearish")
        .otherwise("neutral")
    ).withColumn(
        "momentum_indicator",
        when((coalesce(col("macd_histogram"), lit(0)) > 0) & (coalesce(col("rsi_14d"), lit(50)) > 50), "positive")
        .when((coalesce(col("macd_histogram"), lit(0)) < 0) & (coalesce(col("rsi_14d"), lit(50)) < 50), "negative")
        .otherwise("neutral")
    ).withColumn(
        "completeness_score",
        when(col("sma_20d").isNotNull() & col("rsi_14d").isNotNull() & col("volume_ma_20d").isNotNull(), 1.0)
        .when(col("sma_5d").isNotNull() & col("volume").isNotNull(), 0.7)
        .otherwise(0.3)
    ).withColumn(
        "reliability_score",
        when(coalesce(col("volume"), lit(0)) > 100000, 1.0)
        .when(coalesce(col("volume"), lit(0)) > 10000, 0.8)
        .otherwise(0.5)
    )
    
    print("✅ Comprehensive technical indicators calculated with all required fields")
    return final_df

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Main Combined Processing Pipeline

# COMMAND ----------

def cleanup_existing_silver_tables():
    """Clean up existing silver tables to avoid schema conflicts"""
    try:
        print("🧹 Cleaning up existing Silver tables for fresh schema...")
        
        # Drop existing tables if they exist
        tables_to_clean = [silver_stock_table, silver_news_table]
        
        for table_name in tables_to_clean:
            try:
                spark.sql(f"DROP TABLE IF EXISTS {table_name}")
                print(f"✅ Dropped existing table: {table_name}")
            except Exception as e:
                print(f"⚠️ Note: Could not drop {table_name}: {e}")
        
        print("✅ Silver tables cleanup completed")
        
    except Exception as e:
        print(f"⚠️ Cleanup warning (proceeding anyway): {e}")

def process_combined_bronze_to_silver():
    """Combined pipeline with explicit schemas and comprehensive processing"""
    
    batch_id = f"silver_combined_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    start_time = datetime.now()
    
    print(f"\n🥈 Starting Combined Silver Layer Processing")
    print(f"📋 Batch ID: {batch_id}")
    print(f"⏰ Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🎯 Features: Explicit schemas + FinBERT + Technical indicators")
    
    processing_metrics = {
        'batch_id': batch_id,
        'processing_date': datetime.now().date(),
        'bronze_stock_records': 0,
        'bronze_news_records': 0,
        'silver_stock_records': 0,
        'silver_news_records': 0,
        'processing_errors': []
    }
    
    try:
        # Phase 1: Enhanced Stock Processing
        print(f"\n📊 Phase 1: Enhanced Stock Processing")
        
        try:
            bronze_stock_df = spark.table(bronze_stock_table)
            stock_count = bronze_stock_df.count()
            processing_metrics['bronze_stock_records'] = stock_count
            print(f"📈 Processing {stock_count:,} stock records")
            
            if stock_count > 0:
                # Apply technical indicators
                enhanced_stock_df = calculate_comprehensive_technical_indicators(bronze_stock_df)
                
                # Prepare with explicit schema compliance
                final_stock_df = enhanced_stock_df.select(
                    col("symbol").cast("string"),
                    col("date").cast("date"),
                    col("timestamp").cast("timestamp"),
                    col("open_price").cast("double"),
                    col("high_price").cast("double"), 
                    col("low_price").cast("double"),
                    col("close_price").cast("double"),
                    # Ensure adjusted_close is always double type
                    coalesce(col("adjusted_close"), col("close_price")).cast("double").alias("adjusted_close"),
                    col("volume").cast("bigint"),
                    coalesce(col("split_coefficient"), lit(1.0)).cast("double").alias("split_coefficient"),
                    coalesce(col("dividend_amount"), lit(0.0)).cast("double").alias("dividend_amount"),
                    
                    # Price indicators with explicit casting
                    coalesce(col("price_change"), lit(0.0)).cast("double").alias("price_change"),
                    coalesce(col("price_change_pct"), lit(0.0)).cast("double").alias("price_change_pct"),
                    coalesce(col("daily_range"), lit(0.0)).cast("double").alias("daily_range"),
                    coalesce(col("daily_range_pct"), lit(0.0)).cast("double").alias("daily_range_pct"),
                    coalesce(col("gap_pct"), lit(0.0)).cast("double").alias("gap_pct"),
                    
                    # Volume indicators with explicit casting
                    coalesce(col("volume_change"), lit(0)).cast("bigint").alias("volume_change"),
                    coalesce(col("volume_change_pct"), lit(0.0)).cast("double").alias("volume_change_pct"),
                    coalesce(col("volume_ma_5d"), lit(0.0)).cast("double").alias("volume_ma_5d"),
                    coalesce(col("volume_ma_20d"), lit(0.0)).cast("double").alias("volume_ma_20d"),
                    coalesce(col("volume_ratio"), lit(1.0)).cast("double").alias("volume_ratio"),
                    
                    # Moving averages with explicit casting
                    coalesce(col("sma_5d"), lit(0.0)).cast("double").alias("sma_5d"),
                    coalesce(col("sma_10d"), lit(0.0)).cast("double").alias("sma_10d"),
                    coalesce(col("sma_20d"), lit(0.0)).cast("double").alias("sma_20d"),
                    coalesce(col("sma_50d"), lit(0.0)).cast("double").alias("sma_50d"),
                    coalesce(col("ema_12d"), lit(0.0)).cast("double").alias("ema_12d"),
                    coalesce(col("ema_26d"), lit(0.0)).cast("double").alias("ema_26d"),
                    
                    # Technical indicators with explicit casting
                    coalesce(col("rsi_14d"), lit(50.0)).cast("double").alias("rsi_14d"),
                    coalesce(col("macd"), lit(0.0)).cast("double").alias("macd"),
                    coalesce(col("macd_signal"), lit(0.0)).cast("double").alias("macd_signal"),
                    coalesce(col("macd_histogram"), lit(0.0)).cast("double").alias("macd_histogram"),
                    coalesce(col("bollinger_upper"), lit(0.0)).cast("double").alias("bollinger_upper"),
                    coalesce(col("bollinger_lower"), lit(0.0)).cast("double").alias("bollinger_lower"),
                    coalesce(col("bollinger_position"), lit(0.5)).cast("double").alias("bollinger_position"),
                    
                    # Volatility measures with explicit casting
                    coalesce(col("volatility_5d"), lit(0.0)).cast("double").alias("volatility_5d"),
                    coalesce(col("volatility_20d"), lit(0.0)).cast("double").alias("volatility_20d"),
                    coalesce(col("atr_14d"), lit(0.0)).cast("double").alias("atr_14d"),
                    
                    # Trend analysis with explicit casting
                    coalesce(col("trend_direction"), lit("sideways")).cast("string").alias("trend_direction"),
                    coalesce(col("trend_strength"), lit(0.0)).cast("double").alias("trend_strength"),
                    coalesce(col("support_level"), lit(0.0)).cast("double").alias("support_level"),
                    coalesce(col("resistance_level"), lit(0.0)).cast("double").alias("resistance_level"),
                    
                    # Signal generation with explicit casting
                    coalesce(col("technical_signal"), lit("hold")).cast("string").alias("technical_signal"),
                    coalesce(col("market_sentiment"), lit("neutral")).cast("string").alias("market_sentiment"),
                    coalesce(col("momentum_indicator"), lit("neutral")).cast("string").alias("momentum_indicator"),
                    
                    # Quality metrics with explicit casting
                    coalesce(col("data_quality_score"), lit(1.0)).cast("double").alias("data_quality_score"),
                    coalesce(col("completeness_score"), lit(0.8)).cast("double").alias("completeness_score"),
                    coalesce(col("reliability_score"), lit(0.8)).cast("double").alias("reliability_score"),
                    
                    # Processing metadata with explicit casting
                    lit(bronze_stock_table).cast("string").alias("source"),
                    current_timestamp().alias("processing_timestamp"),
                    lit(batch_id).cast("string").alias("ingestion_batch"),
                    current_date().alias("processed_date")
                )
                
                # Type-safe save with overwrite mode to avoid schema conflicts
                final_stock_df.write \
                    .format("delta") \
                    .mode("overwrite") \
                    .option("mergeSchema", "false") \
                    .option("overwriteSchema", "true") \
                    .saveAsTable(silver_stock_table)
                
                processing_metrics['silver_stock_records'] = final_stock_df.count()
                print(f"✅ Created {processing_metrics['silver_stock_records']:,} enhanced stock records")
            
        except Exception as e:
            error_msg = f"Stock processing error: {str(e)}"
            processing_metrics['processing_errors'].append(error_msg)
            print(f"❌ {error_msg}")
        
        # Phase 2: Enhanced News Processing with FinBERT
        print(f"\n🤖 Phase 2: Enhanced News Processing with FinBERT")
        
        try:
            bronze_news_df = spark.table(bronze_news_table)
            news_count = bronze_news_df.count()
            processing_metrics['bronze_news_records'] = news_count
            print(f"📰 Found {news_count:,} news records")
            
            if news_count > 0:
                # Filter and limit for stable processing
                filtered_df = bronze_news_df.filter(
                    (col("title").isNotNull()) & 
                    (length(col("title")) > 5)
                ).limit(SILVER_CONFIG['max_news_records_per_batch'])
                
                records = filtered_df.collect()
                print(f"🔄 Processing {len(records)} high-quality news records with FinBERT...")
                
                processed_records = []
                
                for i, record in enumerate(records):
                    try:
                        # Handle record attributes safely
                        try:
                            title = record['title'] if 'title' in record else record.title if hasattr(record, 'title') else ""
                            description = record['description'] if 'description' in record else record.description if hasattr(record, 'description') else ""
                            content = record['content'] if 'content' in record else record.content if hasattr(record, 'content') else ""
                            
                            # Safely access other fields
                            article_id = record['article_id'] if 'article_id' in record else record.article_id if hasattr(record, 'article_id') else f"combined_{i}"
                            url = record['url'] if 'url' in record else record.url if hasattr(record, 'url') else ""
                            source = record['source'] if 'source' in record else record.source if hasattr(record, 'source') else "unknown"
                            author = record['author'] if 'author' in record else record.author if hasattr(record, 'author') else "unknown"
                            
                            # Handle readability and financial relevance scores
                            try:
                                readability_score = float(record['readability_score']) if 'readability_score' in record and record['readability_score'] is not None else 0.5
                            except:
                                readability_score = 0.5
                                
                            try:
                                financial_relevance_score = float(record['financial_relevance_score']) if 'financial_relevance_score' in record and record['financial_relevance_score'] is not None else 0.5
                            except:
                                financial_relevance_score = 0.5
                                
                        except Exception as attr_error:
                            print(f"⚠️ Record attribute access error for record {i}: {attr_error}")
                            continue
                        full_text = f"{title}. {description}. {content}"
                        
                        # Apply FinBERT
                        finbert_result = apply_production_finbert_sentiment(full_text)
                        
                        # Handle timestamp parsing safely
                        try:
                            published_at_raw = record['published_at'] if 'published_at' in record else record.published_at if hasattr(record, 'published_at') else None
                            if published_at_raw:
                                if isinstance(published_at_raw, str):
                                    published_at = datetime.fromisoformat(published_at_raw.replace('Z', '+00:00'))
                                else:
                                    published_at = published_at_raw
                                published_date = published_at.date() if hasattr(published_at, 'date') else datetime.now().date()
                            else:
                                published_at = datetime.now(timezone.utc)
                                published_date = datetime.now().date()
                        except Exception as time_error:
                            published_at = datetime.now(timezone.utc)
                            published_date = datetime.now().date()
                        
                        # Create record with explicit type matching
                        processed_record = (
                            article_id[:200],  # article_id - truncated
                            title[:500],  # title
                            description[:1000],  # description
                            content[:2000],  # content
                            url[:500],  # url
                            source[:100],  # source
                            author[:200],  # author
                            published_at,  # published_at
                            published_date,  # published_date
                            
                            len(content),  # content_length
                            len(title),  # title_length
                            readability_score,  # readability_score
                            financial_relevance_score,  # financial_relevance_score
                            
                            [],  # mentioned_symbols
                            [],  # financial_entities
                            [],  # key_financial_terms
                            [],  # named_entities
                            
                            {},  # original_sentiment_indicators
                            
                            finbert_result['finbert_label'],  # finbert_label
                            float(finbert_result['finbert_score']),  # finbert_score
                            float(finbert_result['finbert_confidence']),  # finbert_confidence
                            float(finbert_result['finbert_negative']),  # finbert_negative
                            float(finbert_result['finbert_neutral']),  # finbert_neutral
                            float(finbert_result['finbert_positive']),  # finbert_positive
                            finbert_result['processing_method'],  # finbert_processing_method
                            float(finbert_result['processing_time_ms']),  # finbert_processing_time_ms
                            
                            'medium',  # sentiment_intensity
                            float(python_abs(finbert_result['finbert_score'])),  # sentiment_subjectivity
                            finbert_result['finbert_label'],  # emotional_tone
                            0.5,  # urgency_score
                            
                            'general',  # news_category
                            'medium',  # market_impact_category
                            'normal',  # time_sensitivity
                            
                            0.8,  # content_quality_score
                            float(finbert_result['finbert_confidence']),  # sentiment_quality_score
                            0.8,  # overall_reliability_score
                            
                            True,  # market_hours_flag
                            True,  # weekday_flag
                            False,  # earnings_season_flag
                            
                            datetime.now(timezone.utc),  # processing_timestamp
                            bronze_news_table,  # data_source
                            batch_id,  # ingestion_batch
                            datetime.now().date()  # processed_date
                        )
                        
                        processed_records.append(processed_record)
                        
                        if (i + 1) % 10 == 0:
                            print(f"🔄 Processed {i + 1}/{len(records)} ({((i + 1)/len(records)*100):.1f}%)")
                            
                    except Exception as e:
                        print(f"⚠️ Error processing record {i}: {str(e)}")
                        continue
                
                # Create DataFrame with explicit schema
                if processed_records:
                    print(f"💾 Creating DataFrame with explicit schema for {len(processed_records)} records...")
                    
                    news_df = spark.createDataFrame(processed_records, schema=get_enhanced_news_schema())
                    
                    # Type-safe save with overwrite to avoid schema conflicts
                    news_df.write \
                        .format("delta") \
                        .mode("overwrite") \
                        .option("mergeSchema", "false") \
                        .option("overwriteSchema", "true") \
                        .saveAsTable(silver_news_table)
                    
                    processing_metrics['silver_news_records'] = len(processed_records)
                    print(f"✅ Created {len(processed_records):,} enhanced news records")
        
        except Exception as e:
            error_msg = f"News processing error: {str(e)}"
            processing_metrics['processing_errors'].append(error_msg)
            print(f"❌ {error_msg}")
        
        # Phase 3: Optimization
        print(f"\n⚡ Phase 3: Table Optimization")
        
        try:
            for table_name in [silver_stock_table, silver_news_table]:
                try:
                    table_count = spark.table(table_name).count()
                    if table_count > 0:
                        spark.sql(f"OPTIMIZE {table_name}")
                        print(f"✅ Optimized {table_name} ({table_count:,} records)")
                except Exception as opt_error:
                    print(f"⚠️ Optimization warning: {opt_error}")
                    
        except Exception as e:
            print(f"⚠️ Optimization phase warning: {e}")
    
    except Exception as e:
        error_msg = f"Pipeline error: {str(e)}"
        processing_metrics['processing_errors'].append(error_msg)
        print(f"❌ {error_msg}")
    
    finally:
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds() / 60
        
        print_combined_summary(processing_metrics, start_time, end_time, duration)
    
    return processing_metrics

def print_combined_summary(metrics, start_time, end_time, duration_minutes):
    """Print comprehensive processing summary"""
    
    print(f"\n{'='*80}")
    print(f"🥈 SILVER LAYER PROCESSING SUMMARY")
    print(f"{'='*80}")
    
    print(f"📋 Processing Details:")
    print(f"   Batch ID: {metrics['batch_id']}")
    print(f"   Duration: {duration_minutes:.2f} minutes")
    print(f"   Start: {start_time.strftime('%H:%M:%S')}")
    print(f"   End: {end_time.strftime('%H:%M:%S')}")
    
    total_silver = metrics['silver_stock_records'] + metrics['silver_news_records']
    
    print(f"\n📊 Processing Results:")
    print(f"   📈 Stock Records: {metrics['bronze_stock_records']:,} → {metrics['silver_stock_records']:,}")
    print(f"   📰 News Records: {metrics['bronze_news_records']:,} → {metrics['silver_news_records']:,}")
    print(f"   🎯 Total Enhanced: {total_silver:,} records")
    
    print(f"\n🤖 FinBERT Processing:")
    print(f"   Model Status: {'Production' if finbert_ready else 'Enhanced Fallback'}")
    print(f"   Total Calls: {processing_stats['finbert_calls']:,}")
    print(f"   Avg Time: {processing_stats.get('avg_time_per_call', 0)*1000:.2f}ms")
    
    print(f"\n🎯 Enhanced Features Created:")
    print(f"   ✅ Technical indicators (RSI, MACD, Bollinger Bands)")
    print(f"   ✅ FinBERT sentiment analysis with confidence")
    print(f"   ✅ Type-safe DataFrame creation with explicit schemas")
    print(f"   ✅ Quality scoring and reliability metrics")
    print(f"   ✅ Optimized for correlation analysis")
    
    if metrics['processing_errors']:
        print(f"\n⚠️ Processing Issues ({len(metrics['processing_errors'])}):")
        for error in metrics['processing_errors'][:2]:
            print(f"   • {error}")
    
    if total_silver > 0:
        print(f"\n✅ STATUS: SUCCESS - Ready for Gold layer correlation analysis!")
    else:
        print(f"\n⚠️ STATUS: Review Bronze layer data availability")
    
    print(f"{'='*80}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. Execute Combined Processing Pipeline

# COMMAND ----------

# Execute the combined processing
try:
    print(f"🚀 Launching Silver Layer Processing")
    print(f"🎯 Key Features:")
    print(f"   • Explicit schemas for type safety")
    print(f"   • Production FinBERT sentiment analysis")
    print(f"   • Comprehensive technical indicators")
    print(f"   • Enhanced error handling and stability")
    
    # Clean up existing tables first
    cleanup_existing_silver_tables()
    
    processing_results = process_combined_bronze_to_silver()
    
    print(f"\n🎉 Combined Silver Layer Processing Complete!")
    
    # Final verification
    try:
        stock_count = spark.table(silver_stock_table).count() if spark.catalog.tableExists(silver_stock_table) else 0
        news_count = spark.table(silver_news_table).count() if spark.catalog.tableExists(silver_news_table) else 0
        
        print(f"\n📊 Final Silver Layer Status:")
        print(f"   📈 Enhanced Stock Records: {stock_count:,}")
        print(f"   📰 Enhanced News Records: {news_count:,}")
        
        if stock_count > 0 and news_count > 0:
            print(f"\n🔗 Correlation Analysis Ready:")
            print(f"   ✅ Stock technical indicators available")
            print(f"   ✅ News sentiment analysis complete")
            print(f"   ✅ Both datasets in Silver layer")
            print(f"   🚀 Ready for Gold layer processing!")
            
            # Show preview of correlation readiness
            print(f"\n📊 Stock Data Preview:")
            spark.table(silver_stock_table) \
                .select("symbol", "date", "close_price", "rsi_14d", "technical_signal") \
                .orderBy("date", "symbol") \
                .limit(3) \
                .show()
            
            print(f"\n📰 News Data Preview:")
            spark.table(silver_news_table) \
                .select("title", "finbert_label", "finbert_score", "finbert_confidence") \
                .orderBy(col("processing_timestamp").desc()) \
                .limit(3) \
                .show(truncate=False)
                
    except Exception as verification_error:
        print(f"⚠️ Verification warning: {verification_error}")
    
except Exception as e:
    print(f"❌ Combined processing execution failed: {e}")
    print(f"📋 Full error traceback:")
    traceback.print_exc()

# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Final Verification and Readiness Assessment

# COMMAND ----------

def final_silver_verification():
    """Comprehensive verification of Silver layer readiness"""
    
    print(f"\n🔍 Final Silver Layer Verification")
    print(f"{'='*60}")
    
    verification_results = {
        'stock_ready': False,
        'news_ready': False,
        'correlation_ready': False,
        'quality_score': 0.0
    }
    
    try:
        # Stock data verification
        print(f"\n📊 Stock Data Verification:")
        try:
            stock_df = spark.table(silver_stock_table)
            stock_count = stock_df.count()
            
            if stock_count > 0:
                # Check technical indicators
                tech_completeness = stock_df.agg(
                    avg(when(col("rsi_14d").isNotNull(), 1.0).otherwise(0.0)).alias("rsi_complete"),
                    avg(when(col("macd").isNotNull(), 1.0).otherwise(0.0)).alias("macd_complete"),
                    avg(when(col("bollinger_upper").isNotNull(), 1.0).otherwise(0.0)).alias("bb_complete")
                ).collect()[0]
                
                stock_quality = (tech_completeness['rsi_complete'] + 
                               tech_completeness['macd_complete'] + 
                               tech_completeness['bb_complete']) / 3
                
                print(f"   📈 Records: {stock_count:,}")
                print(f"   📊 Technical Indicators Quality: {stock_quality:.2%}")
                
                if stock_quality > 0.7:
                    verification_results['stock_ready'] = True
                    print(f"   ✅ Stock data ready for correlation analysis")
                else:
                    print(f"   ⚠️ Stock data quality needs improvement")
            else:
                print(f"   ❌ No stock records found")
                
        except Exception as e:
            print(f"   ❌ Stock verification error: {e}")
        
        # News data verification
        print(f"\n📰 News Data Verification:")
        try:
            news_df = spark.table(silver_news_table)
            news_count = news_df.count()
            
            if news_count > 0:
                # Check FinBERT completeness
                sentiment_stats = news_df.agg(
                    avg(when(col("finbert_confidence") >= 0.6, 1.0).otherwise(0.0)).alias("high_confidence"),
                    countDistinct("finbert_label").alias("sentiment_variety"),
                    avg("finbert_confidence").alias("avg_confidence")
                ).collect()[0]
                
                print(f"   📰 Records: {news_count:,}")
                print(f"   🤖 High Confidence Predictions: {sentiment_stats['high_confidence']:.2%}")
                print(f"   📊 Average FinBERT Confidence: {sentiment_stats['avg_confidence']:.3f}")
                print(f"   🎯 Sentiment Variety: {sentiment_stats['sentiment_variety']} labels")
                
                if sentiment_stats['high_confidence'] > 0.5 and sentiment_stats['sentiment_variety'] >= 2:
                    verification_results['news_ready'] = True
                    print(f"   ✅ News data ready for correlation analysis")
                else:
                    print(f"   ⚠️ News data quality needs improvement")
                    
            else:
                print(f"   ❌ No news records found")
                
        except Exception as e:
            print(f"   ❌ News verification error: {e}")
        
        # Correlation readiness assessment
        print(f"\n🔗 Correlation Analysis Readiness:")
        
        if verification_results['stock_ready'] and verification_results['news_ready']:
            verification_results['correlation_ready'] = True
            verification_results['quality_score'] = 0.9
            
            print(f"   ✅ Both datasets are ready for correlation analysis")
            print(f"   ✅ Technical indicators calculated and validated")
            print(f"   ✅ FinBERT sentiment analysis completed")
            print(f"   ✅ Data quality thresholds met")
            
        else:
            print(f"   ⚠️ Prerequisites not fully met for correlation analysis")
            if not verification_results['stock_ready']:
                print(f"   • Stock data quality needs improvement")
            if not verification_results['news_ready']:
                print(f"   • News sentiment analysis needs improvement")
        
        print(f"\n📊 Overall Silver Layer Quality Score: {verification_results['quality_score']:.1%}")
        
    except Exception as e:
        print(f"❌ Verification failed: {e}")
        traceback.print_exc()
    
    return verification_results

# Run final verification
verification_results = final_silver_verification()

print(f"\n🎊 SILVER LAYER PROCESSOR COMPLETED")
print(f"{'='*80}")

if verification_results['correlation_ready']:
    print(f"🎯 STATUS: READY FOR ADVANCED ANALYTICS")
else:
    print(f"⚠️ STATUS: REVIEW REQUIRED")
    print(f"💡 Check data quality and processing configuration")

print(f"\n⏰ Processing completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# COMMAND ----------

🥈 Silver Layer Processor - Complete Combined Version
⏰ Started: 2025-07-21 22:23:12
🎯 Focus: Bronze → Silver with explicit schemas and comprehensive analytics
📋 Current catalog: databricks_stock_sentiment_canada
✅ Configuration loaded successfully
📊 Bronze Stock Table: databricks_stock_sentiment_canada.bronze.historical_stock_data
📰 Bronze News Table: databricks_stock_sentiment_canada.bronze.historical_news_data
📊 Enhanced processing configuration:
   batch_size: 1000
   finbert_confidence_threshold: 0.6
   data_quality_threshold: 0.7
   technical_indicators_window: 20
   sentiment_aggregation_window: 7
   correlation_analysis_enabled: True
   feature_engineering_enabled: True
   max_news_records_per_batch: 50
   enable_explicit_schema: True
✅ Explicit schemas defined for type-safe DataFrame creation
📊 Stock schema: 51 fields
📰 News schema: 43 fields
🤖 Loading enhanced FinBERT model...


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ FinBERT model ready for production
🤖 FinBERT Status: Production Ready
🏗️ Creating enhanced Silver layer tables...
✅ Silver schema created in catalog: databricks_stock_sentiment_canada
✅ Created enhanced stock table: databricks_stock_sentiment_canada.silver.enhanced_stock_data
✅ Created enhanced news table: databricks_stock_sentiment_canada.silver.enhanced_news_data
✅ Enhanced Silver layer tables created successfully
🚀 Launching Silver Layer Processing
🎯 Key Features:
   • Explicit schemas for type safety
   • Production FinBERT sentiment analysis
   • Comprehensive technical indicators
   • Enhanced error handling and stability
🧹 Cleaning up existing Silver tables for fresh schema...
✅ Dropped existing table: databricks_stock_sentiment_canada.silver.enhanced_stock_data
✅ Dropped existing table: databricks_stock_sentiment_canada.silver.enhanced_news_data
✅ Silver tables cleanup completed

🥈 Starting Combined Silver Layer Processing
📋 Batch ID: silver_combined_20250721_222342
⏰ Started

In [0]:
%python
import json
from datetime import datetime

# Airflow Integration - Success/Failure Reporting

try:
    # If we reach here, notebook executed successfully
    success_result = {
        "status": "SUCCESS",
        "message": "Notebook execution completed successfully",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "records_processed": locals().get('total_records_processed', 0),  # Update based on your variables
        "data_quality_score": locals().get('data_quality_score', 1.0)     # Update based on your variables
    }
    
    print(f"✅ Notebook Success:")
    print(json.dumps(success_result, indent=2))
    
    # Exit with success status for Airflow
    dbutils.notebook.exit(success_result)
    
except Exception as e:
    # If any error occurs, report failure
    failure_result = {
        "status": "FAILED", 
        "message": f"Notebook execution failed: {str(e)}",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "error_type": type(e).__name__
    }
    
    print(f"❌ Notebook Failure:")
    print(json.dumps(failure_result, indent=2))
    
    # Exit with failure status for Airflow
    dbutils.notebook.exit(failure_result)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-7398733180968179>, line 11
      6 try:
      7     # If we reach here, notebook executed successfully
      8     success_result = {
      9         "status": "SUCCESS",
     10         "message": "Notebook execution completed successfully",
---> 11         "batch_id": batch_id,
     12         "execution_timestamp": datetime.now().isoformat(),
     13         "records_processed": locals().get('total_records_processed', 0),  # Update based on your variables
     14         "data_quality_score": locals().get('data_quality_score', 1.0)     # Update based on your variables
     15     }
     17     print(f"✅ Notebook Success:")

NameError: name 'batch_id' is not defined

During handling of the above exception, another exception occurred:

NameError                                 Traceback (most recent call last)
File <comma